Libraries:

In [14]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras import backend as K
import os

Set theano backend for images and change the directory

In [15]:
K.set_image_dim_ordering('th')
os.chdir("/home/bombus/Project/Project/Data/ISIC_MSK-2_1_sorted")

Set paths and variables

In [16]:
# dimensions of our images.
img_width, img_height = 150, 150
top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'train'
validation_data_dir = 'validation'
test_dir = 'test'
nb_train_samples = 1986
nb_validation_samples = 778
epochs = 50
batch_size = 16

Define all functions

In [17]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save('bottleneck_features_train',
        bottleneck_features_train)
    print("done saving train features")
    
    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save('bottleneck_features_validation',
            bottleneck_features_validation)
    print("done saving validation features")

Create and train the fully-connected layers

In [18]:
def train_top_model():
    train_data = np.load('bottleneck_features_train.npy')
    train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load('bottleneck_features_validation.npy')
    validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])
    
    print("starting fitting")    
    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    
    model.save_weights(top_model_weights_path)
    print("ended fitting and saving")

Predict from test data

In [19]:
def predict_image_class(file):
    model = applications.VGG16(include_top=False, weights='imagenet')
    x = load_img(file, target_size=(img_width,img_height))
    x = img_to_array(x)
    x = np.expand_dims(x, axis=0)
    array = model.predict(x)
    model = Sequential()
    model.add(Flatten(input_shape=array.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.load_weights(top_model_weights_path)
    class_predicted = model.predict_classes(array)
    if class_predicted==1:
        print("dogs")
    else:
        print("cat")

Call the functions

In [20]:
save_bottlebeck_features()

/home/bombus/anaconda3/lib/python3.6/site-packages/keras/applications/vgg16.py:181: UserWarning: You are using the TensorFlow backend, yet you are using the Theano image data format convention (`image_data_format="channels_first"`). For best performance, set `image_data_format="channels_last"` in your Keras config at ~/.keras/keras.json.
  warnings.warn('You are using the TensorFlow backend, yet you '


Found 841 images belonging to 2 classes.
done saving train features
Found 373 images belonging to 2 classes.
done saving validation features


In [13]:
train_top_model()

starting fitting
Train on 1986 samples, validate on 778 samples
Epoch 1/50
1986/1986 [==============================] - 2s - loss: 0.8423 - acc: 0.4909 - val_loss: 0.6937 - val_acc: 0.5000
Epoch 2/50
1986/1986 [==============================] - 2s - loss: 0.6976 - acc: 0.4859 - val_loss: 0.6934 - val_acc: 0.5000
Epoch 3/50
1986/1986 [==============================] - 2s - loss: 0.6940 - acc: 0.4970 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 4/50
1986/1986 [==============================] - 2s - loss: 0.6942 - acc: 0.4930 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 5/50
1986/1986 [==============================] - 2s - loss: 0.6935 - acc: 0.4945 - val_loss: 0.6932 - val_acc: 0.5013
Epoch 6/50
1986/1986 [==============================] - 2s - loss: 0.6941 - acc: 0.4879 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 7/50
1986/1986 [==============================] - 2s - loss: 0.6934 - acc: 0.4935 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 8/50
1986/1986 [==============================] - 2s - 

In [21]:
test_files = os.listdir(test_dir)
predict_image_class(test_dir + "test_files[0]")
#predict_image_class(test_dir + "/dogTest.jpg")

/home/bombus/anaconda3/lib/python3.6/site-packages/keras/applications/vgg16.py:181: UserWarning: You are using the TensorFlow backend, yet you are using the Theano image data format convention (`image_data_format="channels_first"`). For best performance, set `image_data_format="channels_last"` in your Keras config at ~/.keras/keras.json.
  warnings.warn('You are using the TensorFlow backend, yet you '


FileNotFoundError: [Errno 2] No such file or directory: 'testtest_files[0]'